# AMA - Progetto Fabbisogni - Emesso Non Riscosso

# INIT

In [ ]:
%run Init.ipynb


Repo name: AMA_Progetto_Fabbisogni
Repo path: /home/bifani/GitHub/AMA_Progetto_Fabbisogni



<IPython.core.display.Javascript object>

In [ ]:
import datetime

import numpy as np
import pandas as pd

from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from src.datafactory import DataFactory
from src.miscellaneous import *

from src.enr import *

<IPython.core.display.Javascript object>

# DATA

## SQLITE

In [ ]:
!grep -e "SQLITE_DB}" ../src/enr.py | sort

In [ ]:
DataFactory(f"{BASE_FOLDER_ENR}/{SQLITE_DB} | ", dry=True)

## DOM

### CONTRATTI_ATTIVI

In [ ]:
dom_ca = ENR_DOM_CONTRATTI_ATTIVI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
dom_ca.print_counts("CNT_TCN_COD")

### AVVISI_CONTRATTI_ATTIVI

In [ ]:
dom_aca = ENR_DOM_AVVISI_CONTRATTI_ATTIVI(
    # nrows=1000,
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
dom_aca.df["IMPORTO_CONTRATTO"].sum()

In [ ]:
dom_aca.print_counts("MOTIVO_PAREGGIO")

### MERGE

In [ ]:
dom = ENR_DOM(empty=True)

In [ ]:
dom.merge_ALL(step1=True)

In [ ]:
dom.save("DOM")

In [ ]:
dom = ENR_DOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
dom.merge_ALL(step2=True)

In [ ]:
dom.save("DOM")

#### SPLIT

In [ ]:
dom = ENR_DOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
dom.merge_CONTRATTI()

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_GE_VIE()

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_GET()

In [ ]:
dom.print_counts(["FLAG_GET_ATTIVITA", "FLAG_GET_PRATICHE"])

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_DECESSI()

In [ ]:
dom.print_counts("FLAG_DECEDUTO")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_CESSAZIONI()

In [ ]:
dom.print_counts(["FLAG_CESSATA", "AT_FLG_STA_PI", "RI_CLEAN_STA_PAR_IVA"])

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_RECAPITI()

In [ ]:
dom.print_counts("FLAG_RECAPITO")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_ESENZIONI()

In [ ]:
dom.print_counts(["FLAG_ESENTE_SAP", "FLAG_ESENTE_RIC", "FLAG_ESENTE"])

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_RATEIZZAZIONI()

In [ ]:
dom.print_counts(["FLAG_RATE_SAP", "FLAG_RATE_RIC", "FLAG_RATE"])

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_BOLLETTINI_WEB()

In [ ]:
dom.print_counts("FLAG_BOLLETTINO_WEB")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_BOLLETTINI_CC()

In [ ]:
dom.print_counts("FLAG_BOLLETTINO_CC")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_PAGAMENTI_F24()

In [ ]:
dom.print_counts("FLAG_F24")

In [ ]:
dom.save("DOM")

In [ ]:
dom.save(filename="DOM", ext="csv")

### SUMMARY

In [ ]:
dom = ENR_DOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
dom.print_counts(["CNT_COD", "UTZ_CONTRATTO"])

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "TIPO_DOC_PAREGGIO"],
    #     query="MOTIVO_PAREGGIO.notnull()"
)

In [ ]:
dom.print_flags()

In [ ]:
dom.print_importi_flags(
    columns=["CNT_TCN_COD"],
    query="MOTIVO_PAREGGIO.isnull()",
    #     flags=["FLAG_GIUSTIFICATO"]
)

In [ ]:
dom.print_crosstab(
    ["MOTIVO_PAREGGIO", "TIPO_DOC_PAREGGIO"],
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_GIUSTIFICATO==0) & (FLAG_BOLLETTINO_WEB==1 | FLAG_BOLLETTINO_CC==1)",
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "FLAG_GET_ATTIVITA"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0)",
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "FLAG_GET_PRATICHE"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0)",
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],  # , "FLAG_GET_ATTIVITA", "FLAG_GET_PRATICHE"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1)",
    #     query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1) & ~(FLAG_GET_ATTIVITA==1 & FLAG_GET_PRATICHE==1)",
)

In [ ]:
dom.query_apply(
    "MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1)",
    inplace=True,
)

In [ ]:
dom.df.columns

In [ ]:
cols = [
    "CNT_COD",
    "NUM_FATT",
    "UTZ_CONTRATTO",
    "IMPORTO_CONTRATTO",
    #        'MOTIVO_PAREGGIO', 'DATA_PAREGGIO', 'IMPORTO_PAREGGIO',
    #        'TIPO_DOC_PAREGGIO',
    "CNT_TCN_COD",
    "CNT_COGNOME",
    "CNT_NOME",
    "CNT_RAG_SOC",
    "CNT_PAR_IVA",
    "CNT_COD_FSC",
    "UTZ_TARIFTYP",
    #        'UTZ_VIA_DES', 'UTZ_NUM_CIV', 'SUPERFICIE', 'COMPONENTI', 'CAP',
    #        'COD_ZON_URB', 'NUM_CIR', 'FLAG_DECEDUTO', 'FLAG_CESSATA',
    #     "RI_CLEAN_STA_PAR_IVA",
    #     "STATO",
    "FLAG_GET_ATTIVITA",
    #     "STATO_LAVORAZIONE",
    "FLAG_GET_PRATICHE",
    #     'FLAG_RECAPITO',
    #        'FLAG_ESENTE_SAP', 'FLAG_ESENTE_RIC', 'FLAG_ESENTE', 'FLAG_RATE_SAP',
    #        'NUM_RATE', 'IMPORTO_RATEIZZATO', 'FLAG_RATE_RIC', 'FLAG_RATE',
    #        'FLAG_BOLLETTINO_WEB', 'FLAG_BOLLETTINO_CC', 'FLAG_NON_RECAPITATO',
    #        'FLAG_DATI_ERRATI', 'FLAG_NOTIFICATO', 'FLAG_GIUSTIFICATO',
    #        'FLAG_PAGATO'
]

In [ ]:
dom.set_columns(cols)

In [ ]:
dom.df

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
)

In [ ]:
dom.save("DOM", ext="csv")

In [ ]:
dom.groupby(
    ["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
    ["SUPERFICIE", "COMPONENTI"],
    ["mean", "median", "min", "max", "std"],
)

In [ ]:
dom.print_counts("COMPONENTI")

#### MUNICIPI

In [ ]:
dom.df.boxplot(column="IMPORTO_CONTRATTO", by="NUM_CIR")

In [ ]:
sns.displot(
    data=dom.df.query("IMPORTO_CONTRATTO>0 & IMPORTO_CONTRATTO<10000"),
    x="IMPORTO_CONTRATTO",
    hue="NUM_CIR",
    kind="hist",
    bins=100,
    rug=False,
)

In [ ]:
dom.print_municipi()

In [ ]:
x = list()
for i, m in enumerate(dom.df["NUM_CIR"].unique()):
    d = dom.df.query(f"NUM_CIR == '{m}'")["IMPORTO_CONTRATTO"].to_list()
    x.append(d)
    if i == 1:
        break

len(x)

In [ ]:
stats.kruskal(*x)

In [ ]:
sp.posthoc_dunn(x, p_adjust="holm") < 0.05

#### UTZ_TARIFTYP

In [ ]:
dom.print_importi(columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "UTZ_TARIFTYP"], query="MOTIVO_PAREGGIO.isnull()")
# FLAG_ESENTE==0 & FLAG_RATE==0

#### RCP_SGL_NAZ

In [ ]:
dom.print_importi(columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "RCP_SGL_NAZ"], query="MOTIVO_PAREGGIO.isnull()")

## NDOM

### CONTRATTI_ATTIVI

In [ ]:
ndom_ca = ENR_NDOM_CONTRATTI_ATTIVI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_ca.print_counts("CNT_TCN_COD")

### AVVISI_CONTRATTI_ATTIVI

In [ ]:
ndom_aca = ENR_NDOM_AVVISI_CONTRATTI_ATTIVI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_aca.df["IMPORTO_CONTRATTO"].sum()

In [ ]:
ndom_aca.print_counts("MOTIVO_PAREGGIO")

### MERGE

In [ ]:
ndom = ENR_NDOM(empty=True)

In [ ]:
ndom.merge_ALL()

In [ ]:
ndom.columns

In [ ]:
ndom.save("NDOM")

#### SPLIT

In [ ]:
ndom = ENR_NDOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
ndom.merge_CONTRATTI()

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_GE_VIE()

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_GET()

In [ ]:
ndom.print_counts(["FLAG_GET_ATTIVITA", "FLAG_GET_PRATICHE"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_DECESSI()

In [ ]:
ndom.print_counts("FLAG_DECEDUTO")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_CESSAZIONI()

In [ ]:
ndom.print_counts(["FLAG_CESSATA", "AT_FLG_STA_PI", "RI_CLEAN_STA_PAR_IVA"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_PEC()

In [ ]:
ndom.print_counts(["FLAG_PEC_SAP", "FLAG_PEC_INV", "FLAG_NOPEC", "FLAG_PEC"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_ESENZIONI()

In [ ]:
ndom.print_counts(["FLAG_ESENTE_SAP", "FLAG_ESENTE_RIC", "FLAG_ESENTE"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_RATEIZZAZIONI()

In [ ]:
ndom.print_counts(["FLAG_RATE_SAP", "FLAG_RATE_RIC", "FLAG_RATE"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_BOLLETTINI_WEB()

In [ ]:
ndom.print_counts("FLAG_BOLLETTINO_WEB")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_BOLLETTINI_CC()

In [ ]:
ndom.print_counts("FLAG_BOLLETTINO_CC")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_PAGAMENTI_F24()

In [ ]:
ndom.print_counts("FLAG_F24")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.save(filename="NDOM", ext="csv")

### SUMMARY

In [ ]:
ndom = ENR_NDOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
ndom.print_types()

In [ ]:
ndom.print_counts(["CNT_COD", "UTZ_CONTRATTO"])

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "TIPO_DOC_PAREGGIO"],
    #     query="MOTIVO_PAREGGIO.notnull()"
)

In [ ]:
ndom.print_flags()

In [ ]:
ndom.print_importi_flags(
    columns=["CNT_TCN_COD"],
    query="MOTIVO_PAREGGIO.isnull()",
    #     flags=["FLAG_GIUSTIFICATO"]
)

In [ ]:
ndom.print_crosstab(
    ["MOTIVO_PAREGGIO", "TIPO_DOC_PAREGGIO"],
)

In [ ]:
ndom.print_crosstab(
    [
        "FLAG_NON_RECAPITATO",
        "FLAG_DATI_ERRATI",
        "FLAG_RATE",
        "FLAG_ESENTE",
    ],
    query="MOTIVO_PAREGGIO.isnull()",
    split=1,
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_GIUSTIFICATO==0) & (FLAG_BOLLETTINO_WEB==1 | FLAG_BOLLETTINO_CC==1)",
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "FLAG_GET_ATTIVITA"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0)",
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "FLAG_GET_PRATICHE"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0)",
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],  # , "FLAG_GET_ATTIVITA", "FLAG_GET_PRATICHE"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1)",
    #     query="MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1) & ~(FLAG_GET_ATTIVITA==1 & FLAG_GET_PRATICHE==1)",
)

In [ ]:
ndom.query(
    "MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1)"
    #     "MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1) & ~(FLAG_GET_ATTIVITA==1 & FLAG_GET_PRATICHE==1)"
)

In [ ]:
ndom.query_apply(
    "MOTIVO_PAREGGIO.isnull() & (FLAG_GIUSTIFICATO==0) & (FLAG_GET_ATTIVITA==1 | FLAG_GET_PRATICHE==1)",
    inplace=True,
)

In [ ]:
cols = [
    "CNT_COD",
    "NUM_FATT",
    "UTZ_CONTRATTO",
    "IMPORTO_CONTRATTO",
    #        'MOTIVO_PAREGGIO', 'DATA_PAREGGIO', 'IMPORTO_PAREGGIO',
    #        'TIPO_DOC_PAREGGIO',
    "CNT_TCN_COD",
    "CNT_COGNOME",
    "CNT_NOME",
    "CNT_RAG_SOC",
    "CNT_PAR_IVA",
    "CNT_COD_FSC",
    "UTZ_TARIFTYP",
    #        'UTZ_VIA_DES', 'UTZ_NUM_CIV', 'SUPERFICIE',
    #        'CAP', 'COD_ZON_URB',
    #        'NUM_CIR', 'FLAG_DECEDUTO', 'FLAG_CESSATA', 'RI_CLEAN_STA_PAR_IVA',
    #     "STATO",
    "FLAG_GET_ATTIVITA",
    #     "STATO_LAVORAZIONE",
    "FLAG_GET_PRATICHE",
    #        'FLAG_PEC_SAP', 'SEMESTRE', 'FLAG_PEC_INV', 'FLAG_NOPEC', 'FLAG_PEC',
    #        'FLAG_ESENTE_SAP', 'FLAG_ESENTE_RIC', 'FLAG_ESENTE', 'FLAG_RATE_SAP',
    #        'NUM_RATE', 'IMPORTO_RATEIZZATO', 'FLAG_RATE_RIC', 'FLAG_RATE',
    #        'FLAG_BOLLETTINO_WEB', 'FLAG_BOLLETTINO_CC', 'FLAG_NON_RECAPITATO',
    #        'FLAG_DATI_ERRATI', 'FLAG_NOTIFICATO', 'FLAG_GIUSTIFICATO',
    #        'FLAG_PAGATO'
]

In [ ]:
ndom.set_columns(cols)

In [ ]:
ndom.df

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
)

In [ ]:
ndom.save("NDOM", ext="csv")

In [ ]:
ndom.groupby(
    ["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
    ["SUPERFICIE"],
    ["mean", "median", "min", "max", "std"],
)

In [ ]:
ndom.groupby(["NUM_CIR"], ["SUPERFICIE"], ["mean", "median"])

#### MUNICIPI

In [ ]:
ndom.df.boxplot(column="IMPORTO_CONTRATTO", by="NUM_CIR")

In [ ]:
sns.displot(
    data=ndom.df.query("IMPORTO_CONTRATTO>0 & IMPORTO_CONTRATTO<10000"),
    x="IMPORTO_CONTRATTO",
    hue="NUM_CIR",
    kind="hist",
    bins=100,
    rug=False,
)

In [ ]:
ndom.print_municipi()

In [ ]:
x = list()
for i, m in enumerate(ndom.df["NUM_CIR"].unique()):
    d = ndom.df.query(f"NUM_CIR == '{m}'")["IMPORTO_CONTRATTO"].to_list()
    x.append(d)
    if i == 1:
        break

len(x)

In [ ]:
stats.kruskal(*x)

In [ ]:
sp.posthoc_dunn(x, p_adjust="holm") < 0.05

#### UTZ_TARIFTYP

In [ ]:
ndom.print_importi(columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "UTZ_TARIFTYP"], query="MOTIVO_PAREGGIO.isnull()")
# FLAG_ESENTE==0 & FLAG_RATE==0

#### SUPERFICIE

In [ ]:
ndom.df["SUPERFICIE"].describe()

In [ ]:
pd.cut(ndom.df["SUPERFICIE"], 10)

In [ ]:
importo_gb(
    ndom.df.query("IMPORTO_CONTRATTO > 0"),  # .query("SUPERFICIE > 25000"),
    cols=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "SUPERFICIE"],
    query="IMPORTO_DELTA != 0",
    sort=True,
    bins=10,
)

In [ ]:
ndom.query("SUPERFICIE > 50000 & MOTIVO_PAREGGIO.isnull()")

In [ ]:
ndom.query("FLAG_ESENTE == True & FLAG_RATE == True")

In [ ]:
ndom.pivot_table(
    ["FLAG_ESENTE", "FLAG_RATE"],
    "FLAG_PEC_COMB",
    "IMPORTO_CONTRATTO",
    ["sum", "mean"],
)

## CONTRATTI

In [ ]:
cnt = ENR_CONTRATTI()

In [ ]:
cnt.merge_DOM_NDOM()

In [ ]:
cnt.query("CNT_TCN_COD.isnull()")

In [ ]:
cnt.save()

In [ ]:
cnt = ENR_CONTRATTI(
    pickled=True,
    pre_process=False,
    process=False,
    post_process=False,
    verbose=False,
)

In [ ]:
cnt.merge_VOLTURE()

In [ ]:
cnt.print_counts("FLAG_VOLTURA")

In [ ]:
cnt.save()

### SUMMARY

In [ ]:
cnt = ENR_CONTRATTI(
    pickled=True,
)

In [ ]:
cnt.query("CNT_TCN_COD.isnull()")

In [ ]:
cnt.print_counts(
    [
        "TIPO_UTZ",
        #         "CATEGORIA",
        "CNT_TCN_COD",
        "FLAG_DECEDUTO",
        "FLAG_CESSATA",
        "FLAG_PAGATO",
        "FLAG_VOLTURA",
        "FLAG_MODESTA_VALENTI",
    ]
)

In [ ]:
cnt.cross_table(
    [
        "TIPO_UTZ",
        "CNT_TCN_COD",
        "FLAG_PAGATO",
    ],
    [
        "FLAG_DECEDUTO",
        "FLAG_VOLTURA",
        "FLAG_MODESTA_VALENTI",
        "FLAG_CESSATA",
    ],
    query="CNT_TCN_COD.notnull()",
)

In [ ]:
cnt.drop_columns(
    columns=[
        #         "CNT_COD",
        #         "CNT_COGNOME",
        #         "CNT_NOME",
        #         "CNT_RAG_SOC",
        #         "CNT_PAR_IVA",
        #         "CNT_COD_FSC",
        "NUM_FATT",
        "IMPORTO_CONTRATTO",
        "MOTIVO_PAREGGIO",
        "IMPORTO_PAREGGIO",
        "RCP_VIA_DES",
        "UTZ_VIA_DES",
    ]
)

In [ ]:
cnt

In [ ]:
cnt.save("CONTRATTI", ext="csv")

In [ ]:
cnt.drop_columns(
    columns=[
        "CNT_COD",
        "CNT_COGNOME",
        "CNT_NOME",
        "CNT_RAG_SOC",
        "CNT_PAR_IVA",
        "CNT_COD_FSC",
        "NUM_FATT",
        "IMPORTO_CONTRATTO",
        "MOTIVO_PAREGGIO",
        "IMPORTO_PAREGGIO",
        "RCP_VIA_DES",
        "UTZ_VIA_DES",
    ]
)

In [ ]:
cnt

In [ ]:
cnt.save("CONTRATTI_NO_DP", ext="csv")

## FLUSSI

### RECAPITI

In [ ]:
dom_rec = ENR_DOM_RECAPITI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
dom_rec.print_counts("FLAG_RECAPITO")

In [ ]:
dom_rec.print_counts("RCP_CMN_DES")

In [ ]:
dom_rec.df["RCP_CMN_DES"].str.isalpha().value_counts()

In [ ]:
dom_rec.query("RCP_CMN_DES == 'SUSTRI VT'")

In [ ]:
geo = ENR_COMUNI(pre_process=True, process=True, post_process=True, silent=True)

In [ ]:
"MONTERADO" in geo.comuni()

### PEC

In [ ]:
ndom_nopec = ENR_NDOM_PEC_SAP(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_pec = ENR_NDOM_PEC_INVII(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_pec.print_counts("DESCRIZIONE_ERRORE_SEIPEC")

In [ ]:
ndom_pec.print_counts("SEMESTRE")

### GET

In [ ]:
get_att = ENR_GET_ATTIVITA(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
get_att.print_counts(["STATO"])

In [ ]:
get_prt = ENR_GET_PRATICHE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
get_prt.print_counts("STATO_LAVORAZIONE")

In [ ]:
get_prt.df["DATA_PRESENTAZIONE"].min()

In [ ]:
get_prt.print_counts(["STATO_LAVORAZIONE"])

### GE

In [ ]:
ge_t = ENR_GE_TERRIT(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ge_t.print_counts("COD_VIA")

In [ ]:
ge_t.print_counts("NUM_CIR")

In [ ]:
ge_t.print_duplicates(["COD_VIA"])

In [ ]:
ge_t.query("LIM_CIV_DA == LIM_CIV_A")

In [ ]:
ge_t.query("COD_VIA=='100045'")

In [ ]:
ge_t.query("COD_VIA == '1001'")

In [ ]:
ge_t.query("COD_VIA == '99835'")

In [ ]:
ge_v = ENR_GE_VIE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ge_v.merge_GE_TERRIT()

In [ ]:
ge_v.print_duplicates()

In [ ]:
ge_v.print_counts("COD_VIA")

### DECESSI

In [ ]:
dec = ENR_DECESSI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
dec.query("COD_FISCALE == 'CMPNND35L21H501Q'")

In [ ]:
dec.print_counts("VERIFICA_SOGG")

### VOLTURE

In [ ]:
vol = ENR_VOLTURE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
vol.print_dtypes()

In [ ]:
dec.print_counts("VERIFICA_SOGG")

### CESSAZIONI

In [ ]:
ces = ENR_CESSAZIONI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ces.print_counts(["RI_CLEAN_STA_PAR_IVA", "FLAG_CESSATA"])

### ESENZIONI

In [ ]:
ese_sap = ENR_ESENZIONI_SAP(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ese_r = ENR_ESENZIONI_RICHIESTE(
    pre_process=True,
    process=True,
    post_process=True,
)

### RATEIZZAZIONI

In [ ]:
rate_sap = ENR_RATEIZZAZIONI_SAP(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
rate_r = ENR_RATEIZZAZIONI_RICHIESTE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
rate_r.print_counts("Stato")

### RECLAMI

In [ ]:
reclami = ENR_RECLAMI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
reclami.print_counts("STATO")

In [ ]:
reclami.groupby(["CATEGORIA", "CANALEDITRASMISSIONE", "RAGGRUPPAMENTO"], ["ID"], ["count"])

### PAGAMENTI

In [ ]:
pag = ENR_PAGAMENTI(
    pre_process=True,
    process=True,
    post_process=True,
)

### NDOM

In [ ]:
ndom = ENR_NDOM(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom.merge_PAGAMENTI()

In [ ]:
ndom.df["IMPORTO_PAREGGIO"].sum()

In [ ]:
ndom.df["IMPORTO_PAGAMENTO"].sum()

In [ ]:
ndom.query("IMPORTO_PAREGGIO != IMPORTO_PAGAMENTO & IMPORTO_PAGAMENTO != 0")

In [ ]:
ndom2.df["PAREGGIO"] = ~ndom2.df["DATA_PAREGGIO"].isna()
ndom2.df["PAGAMENTO"] = ~ndom2.df["DATA_PAGAMENTO"].isna()

ndom2.print_counts("PAREGGIO")
ndom2.print_counts("PAGAMENTO")

In [ ]:
ndom2.df[(ndom2.df["PAGAMENTO"] == True) & (ndom2.df["PAREGGIO"] == False)]

In [ ]:
ndom2.df[(ndom2.df["PAGAMENTO"] == True) & (ndom2.df["IMPORTO_PAREGGIO"] == 0)]

In [ ]:
(ndom2.df[(ndom2.df["PAGAMENTO"] == True) & (ndom2.df["IMPORTO_PAREGGIO"] == 0)]).value_counts(
    "MOTIVO_PAREGGIO", dropna=False
)

### BONIFICI

In [ ]:
bnf = ENR_PAGAMENTI_BONIFICI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
bnf.groupby(["CCP"], ["IMPORTO_BON"], ["sum"])

In [ ]:
bnf_ = bnf.df.query(
    "NOTE.str.contains('TA[., ]?RI',regex=True,case=False) & NOTE.str.contains('sem',regex=True,case=False)"
)
bnf_ = bnf_.query(
    "~NOTE.str.contains('II',regex=True,case=False) & ~NOTE.str.contains('2o',regex=True,case=False) & ~NOTE.str.contains('[\s]sec',regex=True,case=False)"
)
display(bnf_)
bnf_.groupby("CCP", dropna=False)["IMPORTO_BON"].agg("sum").to_frame()

In [ ]:
note = bnf_["NOTE"]
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    display(note.values)

### F24

In [ ]:
f24 = ENR_PAGAMENTI_F24(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
f24.print_counts("Rateaz")

In [ ]:
sorted(f24.df.value_counts("Rateaz").index)

In [ ]:
f24.groupby(["CodTrib"], ["IMPORTO_F24"], ["sum"])

In [ ]:
f24.query("CodFis=='04069160580'")

### PAGO PA

In [ ]:
rp = ENR_PAGAMENTI_PAGOPA(
    nrows=10000,
    #     dry=True,
    #     verbose=False,
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
rp.groupby(
    ["RETEINCASSO"],
    ["IMPORTO_AVVISO", "IMPORTO_PAGOPA", "IMPORTOTOTALEACCREDITATO"],
    ["sum"],
)

### BOLLETTINI

In [ ]:
bol = ENR_BOLLETTINI_WEB(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
bol.print_duplicates("keys")

In [ ]:
cc = ENR_BOLLETTINI_CC(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
bol.print_duplicates("keys")